In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

# Load iris dataset as an example
iris = load_iris()
X, y = iris.data, iris.target

# Normalize the data
X = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the Ant Colony Optimization class for feature selection
class AntColony:
    def __init__(self, n_ants, n_features, alpha=1.0, beta=2.0, rho=0.5, q0=0.8, iterations=100):
        self.n_ants = n_ants
        self.n_features = n_features
        self.alpha = alpha
        self.beta = beta
        self.rho = rho
        self.q0 = q0
        self.iterations = iterations
        self.pheromone_matrix = np.ones((n_ants, n_features)) / n_features
        self.visibility_matrix = 1.0 / np.random.rand(n_ants, n_features)
        self.feature_indices = np.arange(n_features)

    def select_features(self):
        solutions = []
        for _ in range(self.iterations):
            ant_solutions = []

            for ant in range(self.n_ants):
                selected_features = self._select_features_for_ant(ant)
                ant_solutions.append(selected_features)

            solutions.append(ant_solutions)

            # Update pheromones
            self._update_pheromones(solutions)

        # Choose the best solution
        best_solution = max(solutions[-1], key=lambda x: self._evaluate_solution(x))
        return best_solution

    def _select_features_for_ant(self, ant):
        selected_features = []
        remaining_features = set(self.feature_indices)

        while remaining_features:
            feature = self._select_next_feature(ant, remaining_features)
            selected_features.append(feature)
            remaining_features.remove(feature)

        return selected_features

    def _select_next_feature(self, ant, remaining_features):
        probabilities = self.pheromone_matrix[ant, list(remaining_features)] ** self.alpha * \
                        self.visibility_matrix[ant, list(remaining_features)] ** self.beta

        if np.random.rand() < self.q0:
            # Exploitation: choose the feature with the maximum probability
            return list(remaining_features)[np.argmax(probabilities)]
        else:
            # Exploration: choose a feature based on the probabilities
            probabilities /= probabilities.sum()
            return np.random.choice(list(remaining_features), p=probabilities)

    def _evaluate_solution(self, solution):
        # Evaluate the selected features using a neural network
        clf = MLPClassifier(hidden_layer_sizes=(5,), max_iter=1000, random_state=42)
        clf.fit(X_train[:, solution], y_train)
        predictions = clf.predict(X_test[:, solution])
        return accuracy_score(y_test, predictions)

    def _update_pheromones(self, solutions):
        self.pheromone_matrix *= (1.0 - self.rho)

        for ant, solution in enumerate(solutions[-1]):
            for feature in solution:
                self.pheromone_matrix[ant, feature] += 1.0 / self._evaluate_solution(solution)

# Example usage
n_ants = 10
n_features = X.shape[1]
alpha = 1.0
beta = 2.0
rho = 0.5
q0 = 0.8
iterations = 5

aco = AntColony(n_ants, n_features, alpha, beta, rho, q0, iterations)
selected_features = aco.select_features()

print("Selected Features:", selected_features)

# Evaluate the selected features using a neural network
clf = MLPClassifier(hidden_layer_sizes=(5,), max_iter=1000, random_state=42)
clf.fit(X_train[:, selected_features], y_train)
predictions = clf.predict(X_test[:, selected_features])
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perce

Selected Features: [3, 1, 2, 0]
Accuracy: 0.9666666666666667


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
